In [1]:
import numpy as np 
import pandas as pd

In [2]:
raw_df = pd.read_csv('housing.csv')

In [3]:
cond = ((raw_df['ocean_proximity'] == '<1H OCEAN') | 
        (raw_df['ocean_proximity'] == 'INLAND'))
df = raw_df[cond].copy()

In [4]:
req_cols = ['latitude', 'longitude', 'housing_median_age', 'total_rooms', 
            'total_bedrooms', 'population', 'households', 'median_income', 
            'median_house_value']
df = df[req_cols]

Question 1
There's one feature with missing values. What is it?
- total_rooms
- total_bedrooms
- population
- households

In [5]:
[c for c in df.columns if df[c].isna().sum() > 1]

['total_bedrooms']

Question 2: What's the median (50% percentile) for variable 'population'?
- 995
- 1095
- 1195
- 1295

In [6]:
print('50th percentile of variable population: {}'.format(np.percentile(df['population'], 50)))

50th percentile of variable population: 1195.0


Prepare and split the dataset
- Shuffle the dataset (the filtered one you created above), use seed 42.
- Split your data in train/val/test sets, with 60%/20%/20% distribution.
- Apply the log transformation to the median_house_value variable using the np.log1p() function.

In [7]:
df['median_house_value'] = np.log1p(df['median_house_value'])

In [8]:
np.random.seed(42)
n = len(df)
idx = np.arange(n)
np.random.shuffle(idx)
df_shuffled = df.iloc[idx]

In [9]:
n = len(df_shuffled)
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)
train_df = df_shuffled.iloc[:n_train].copy()
val_df = df_shuffled.iloc[n_train:n_train+n_val].copy()
test_df = df_shuffled.iloc[n_train+n_val:].copy()
print(train_df.shape, val_df.shape, test_df.shape)

(9413, 9) (3137, 9) (3137, 9)


### Linear regression model 

In [10]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X) 
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    return w[0], w[1:]

In [11]:
def train_linear_regression_reg(X, y, r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X)
    reg = r + np.eye(XTX.shape[0])
    XTX = XTX + reg
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    return w[0], w[1:]

In [12]:
def RMSE(y, y_pred):
    error = y - y_pred
    return np.sqrt((error ** 2).mean())

Question 3
- We need to deal with missing values for the column from Q1.
- We have two options: fill it with 0 or with the mean of this variable.
- Try both options. For each, train a linear regression model without regularization using the code from the lessons.
- For computing the mean, use the training only!
- Use the validation dataset to evaluate the models and compare the RMSE of each option.
- Round the RMSE scores to 2 decimal digits using round(score, 2)
- Which option gives better RMSE?
    - With 0
    - With mean
    - Both are equally good

### Option 1: fill missing values with 0 

In [13]:
train_df['total_bedrooms'].fillna(0, inplace=True)
val_df['total_bedrooms'].fillna(0, inplace=True)
test_df['total_bedrooms'].fillna(0, inplace=True)

In [14]:
X = np.array(train_df.loc[:, train_df.columns[:-1]].copy())
Y = np.array(train_df.loc[:, 'median_house_value'])
b, w = train_linear_regression(X, Y)

In [15]:
X_val = np.array(val_df.loc[:, val_df.columns[:-1]].copy())
y_val = np.array(val_df.loc[:, 'median_house_value'])
y_pred_val = b + X_val.dot(w)

In [16]:
round(RMSE(y_val, y_pred_val), 2)

0.34

### Option 2: fill missing value with mean 

In [17]:
n = len(df_shuffled)
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)
train_df = df_shuffled.iloc[:n_train].copy()
val_df = df_shuffled.iloc[n_train:n_train+n_val].copy()
test_df = df_shuffled.iloc[n_train+n_val:].copy()
print(train_df.shape, val_df.shape, test_df.shape)

(9413, 9) (3137, 9) (3137, 9)


In [18]:
avg_value = train_df['total_bedrooms'].mean()
train_df['total_bedrooms'].fillna(avg_value, inplace=True)
val_df['total_bedrooms'].fillna(avg_value, inplace=True)
test_df['total_bedrooms'].fillna(avg_value, inplace=True)

In [19]:
X = np.array(train_df.loc[:, train_df.columns[:-1]].copy())
Y = np.array(train_df.loc[:, 'median_house_value'])
b, w = train_linear_regression(X, Y)

In [20]:
X_val = np.array(val_df.loc[:, val_df.columns[:-1]].copy())
y_val = np.array(val_df.loc[:, 'median_house_value'])
y_pred_val = b + X_val.dot(w)

In [21]:
round(RMSE(y_val, y_pred_val), 2)

0.34

Question 4
- Now let's train a regularized linear regression.
- For this question, fill the NAs with 0.
- Try different values of r from this list: [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10].
- Use RMSE to evaluate the model on the validation dataset.
- Round the RMSE scores to 2 decimal digits.
- Which r gives the best RMSE? If there are multiple options, select the smallest r.
    - 0
    - 0.000001
    - 0.001
    - 0.0001

In [22]:
n = len(df_shuffled)
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)
train_df = df_shuffled.iloc[:n_train].copy()
val_df = df_shuffled.iloc[n_train:n_train+n_val].copy()
test_df = df_shuffled.iloc[n_train+n_val:].copy()
print(train_df.shape, val_df.shape, test_df.shape)

(9413, 9) (3137, 9) (3137, 9)


In [23]:
train_df['total_bedrooms'].fillna(0, inplace=True)
val_df['total_bedrooms'].fillna(0, inplace=True)
test_df['total_bedrooms'].fillna(0, inplace=True)

In [24]:
r_list = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]

In [25]:
X = np.array(train_df.loc[:, train_df.columns[:-1]].copy())
Y = np.array(train_df.loc[:, 'median_house_value'])

In [26]:
X_val = np.array(val_df.loc[:, val_df.columns[:-1]].copy())
y_val = np.array(val_df.loc[:, 'median_house_value'])

In [27]:
rmse_result = {}

In [28]:
for r in r_list:
    b, w = train_linear_regression_reg(X, Y, r)
    y_pred_val = b + X_val.dot(w)
    rmse_result[r] = round(RMSE(y_val, y_pred_val), 2)

In [29]:
best_r = min([k for k, v in rmse_result.items() if v == min(rmse_result.values())])
print('Best r value: {}'.format(best_r))

Best r value: 0


Question 5
- We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
- Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
- For each seed, do the train/validation/test split with 60%/20%/20% distribution.
- Fill the missing values with 0 and train a model without regularization.
- For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
- What's the standard deviation of all the scores? To compute the standard deviation, use np.std.
- Round the result to 3 decimal digits (round(std, 3))
- What's the value of std?
    - 0.5
    - 0.05
    - 0.005
    - 0.0005
- Note: Standard deviation shows how different the values are. If it's low, then all values are approximately the same. If it's high, the values are different. If standard deviation of scores is low, then our model is stable.

In [30]:
seed_list = np.arange(10)

In [31]:
seed_result = {}

In [32]:
for seed in seed_list:
    np.random.seed(seed)
    n = len(df)
    idx = np.arange(n)
    np.random.shuffle(idx)
    df_shuffled = df.iloc[idx]
    train_df = df_shuffled.iloc[:n_train].copy()
    val_df = df_shuffled.iloc[n_train:n_train+n_val].copy()
    test_df = df_shuffled.iloc[n_train+n_val:].copy()
    
    train_df['total_bedrooms'].fillna(0, inplace=True)
    val_df['total_bedrooms'].fillna(0, inplace=True)
    test_df['total_bedrooms'].fillna(0, inplace=True)

    X = np.array(train_df.loc[:, train_df.columns[:-1]].copy())
    Y = np.array(train_df.loc[:, 'median_house_value'])
    X_val = np.array(val_df.loc[:, val_df.columns[:-1]].copy())
    y_val = np.array(val_df.loc[:, 'median_house_value'])
    b, w = train_linear_regression(X, Y)
    y_pred_val = b + X_val.dot(w)
    seed_result[seed] = RMSE(y_val, y_pred_val)

In [33]:
std_value = round(np.std(list(seed_result.values())), 3)
print('Value of std: {}'.format(std_value))

Value of std: 0.005


Question 6
- Split the dataset like previously, use seed 9.
- Combine train and validation datasets.
- Fill the missing values with 0 and train a model with r=0.001.
- What's the RMSE on the test dataset?
    - 0.13
    - 0.23
    - 0.33
    - 0.43

In [34]:
seed = 9
np.random.seed(seed)
n = len(df)
idx = np.arange(n)
np.random.shuffle(idx)
df_shuffled = df.iloc[idx]
train_df = df_shuffled.iloc[:n_train].copy()
val_df = df_shuffled.iloc[n_train:n_train+n_val].copy()
test_df = df_shuffled.iloc[n_train+n_val:].copy()
train_df = pd.concat([train_df, val_df], axis=0)

In [35]:
train_df['total_bedrooms'].fillna(0, inplace=True)
test_df['total_bedrooms'].fillna(0, inplace=True)

In [36]:
X = np.array(train_df.loc[:, train_df.columns[:-1]].copy())
Y = np.array(train_df.loc[:, 'median_house_value'])
X_test = np.array(test_df.loc[:, test_df.columns[:-1]].copy())
y_test = np.array(test_df.loc[:, 'median_house_value'])

In [37]:
r = 0.001
b, w = train_linear_regression_reg(X, Y, r)
y_pred = b + X_test.dot(w)

In [38]:
print('RMSE on test dataset: {}'.format(round(RMSE(y_test, y_pred), 3)))

RMSE on test dataset: 0.338
